## Merge and Join Operations

Merge and join operations come up most often when combining data from different sources. Here we will consider an example of some data about US states and their populations. The data files can be found at http://github.com/jakevdp/data-USstates/:

Pandas implements several of these fundamental building-blocks in the pd.merge( ) function and the related join( ) method of Series and Dataframes. As we will see, these let you efficiently link data from different sources.

In [2]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

### Categories of Joins

The pd.merge() function implements a number of types of joins: the one-to-one, many-to-one, and many-to-many joins. All three types of joins are accessed via an identical call to the pd.merge() interface; the type of join performed depends on the form of the input data. Here we will show simple examples of the three types of merges, and discuss detailed options further below.

#### One-to-one Joins

In [4]:
df1 = pd.DataFrame({'employee': ['Bob','Jake','Lisa','Susan'],
                   'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Susan'],
                   'hire date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')


,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Susan,HR
,employee,hire date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Susan,2014


To combine the information into a single DataFrame, we can use the pd.merge( ) function:
    

In [5]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Susan,HR,2014


The pd.merge( ) function recognizes that each DataFrame has an "employee" column, and automatically joins using this column as a key. The result of the merge is a new DataFrame that combines the information from the two inputs. Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between df1 and df2, and the pd.merge() function correctly accounts for this. Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the left_index and right_index keywords, discussed momentarily).

#### Many-to-one Joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries. For the many-to-one case, the resulting DataFrame will preserve those duplicate entries as appropriate. Consider the following example of a many-to-one join:

In [6]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

display('df3', 'df4', 'pd.merge(df3, df4)')


df3
  employee        group  hire date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3    Susan           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3    Susan           HR       2014      Steve

The resulting DataFrame has an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

#### Many-to-many Joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined. If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge. This will be perhaps most clear with a concrete example. Consider the following, where we have a DataFrame showing one or more skills associated with a particular group. By performing a many-to-many join, we can recover the skills associated with any individual person:

In [7]:
df5 = pd.DataFrame({'group':['Accounting', 'Accounting',
                            'Engineering', 'Engineering', 'HR', 'HR'],
                   'skills': ['math', 'spreadsheets', 'coding', 'linux',
                             'spreadsheets', 'organization']})

display('df1', 'df5', "pd.merge(df1, df5)")


,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Susan,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


### Specification of the Merge Key

Simply, you can explicitly specify the name of the key column using the 'on keyword' which takes a column name or list of column names.

In [13]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")


df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3    Susan           HR

df2
  employee  hire date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3    Susan       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3    Susan           HR       2014

#### The left_on and right_on keywords

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee". In this case, we can use the left_on and right_on keywords to specify the two column names:

In [14]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Susan'],
                   'salary': [70000, 80000, 120000, 90000]})

display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')


df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3    Susan           HR

df3
    name  salary
0    Bob   70000
1   Jake   80000
2   Lisa  120000
3  Susan   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group   name  salary
0      Bob   Accounting    Bob   70000
1     Jake  Engineering   Jake   80000
2     Lisa  Engineering   Lisa  120000
3    Susan           HR  Susan   90000

The result has a redundant column that we can drop if desired–for example, by using the drop( ) method of DataFrames:

In [15]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)


,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Susan,HR,90000


#### The left_index and right_index keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index. For example, your data might look like this:

For convenience, DataFrames implement the join( ) method, which performs a merge that defaults to joining on indices:

In [16]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')


,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Susan,HR
,hire date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [17]:
display('df1a', 'df2a', 'df1a.join(df2a)')


,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Susan,HR
,hire date
employee,
Lisa,2004
Bob,2008
Jake,2012


### Join Mix of Indices and Columns

If you'd like to mix indices and columns, you can combine left_index with right_on or left_on with right_index to get the desired behavior:

In [20]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Susan              HR

df3
    name  salary
0    Bob   70000
1   Jake   80000
2   Lisa  120000
3  Susan   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group   name  salary
0   Accounting    Bob   70000
1  Engineering   Jake   80000
2  Engineering   Lisa  120000
3           HR  Susan   90000

### Specifying Joins with how = 'inner', how = 'left', how = 'right'

In [23]:
df6 = pd.DataFrame ({'name': ['Alice', 'Mike', 'Vita'],
                    'food' : ['salmon', 'pasta', 'salad']},
                   columns = ['name', 'food'])
df7 = pd.DataFrame({'name': ['Mike', 'Gary'],
                   'drink': ['wine', 'beer']},
                  columns = ['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')


df6
    name    food
0  Alice  salmon
1   Mike   pasta
2   Vita   salad

df7
   name drink
0  Mike  wine
1  Gary  beer

pd.merge(df6, df7)
   name   food drink
0  Mike  pasta  wine

Here we have merged two datasets that have only a single "name" entry in common: Mike. By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join. We can specify this explicitly using the how keyword, which defaults to "inner":

In [24]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mike,pasta,wine


Other options for the how keyword are 'outer', 'left', and 'right'. An outer join returns a join over the union of the input columns, and fills in all missing values with NAs:

In [25]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name    food
0  Alice  salmon
1   Mike   pasta
2   Vita   salad

df7
   name drink
0  Mike  wine
1  Gary  beer

pd.merge(df6, df7, how='outer')
    name    food drink
0  Alice  salmon   NaN
1   Mike   pasta  wine
2   Vita   salad   NaN
3   Gary     NaN  beer

In [26]:
display('df6', 'df7', "pd.merge(df6, df7, how='inner')")

df6
    name    food
0  Alice  salmon
1   Mike   pasta
2   Vita   salad

df7
   name drink
0  Mike  wine
1  Gary  beer

pd.merge(df6, df7, how='inner')
   name   food drink
0  Mike  pasta  wine

he left join and right join return joins over the left entries and right entries, respectively. For example:

In [27]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name    food
0  Alice  salmon
1   Mike   pasta
2   Vita   salad

df7
   name drink
0  Mike  wine
1  Gary  beer

pd.merge(df6, df7, how='left')
    name    food drink
0  Alice  salmon   NaN
1   Mike   pasta  wine
2   Vita   salad   NaN

In [29]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
    name    food
0  Alice  salmon
1   Mike   pasta
2   Vita   salad

df7
   name drink
0  Mike  wine
1  Gary  beer

pd.merge(df6, df7, how='right')
   name   food drink
0  Mike  pasta  wine
1  Gary    NaN  beer

### Example: US States Data

In [36]:
# Load the data

filename = '~/Documents/Datasets/data-USstates-Master/state-population.csv'
pop = pd.read_csv(filename)

filename = '~/Documents/Datasets/data-USstates-Master/state-abbrevs.csv'
abbrev = pd.read_csv(filename)

filename = '~/Documents/Datasets/data-USstates-Master/state-areas.csv'
area = pd.read_csv(filename)

display('pop.head()', 'abbrev.head()', 'area.head()')


pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

abbrev.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

area.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density. We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.


Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density. We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population DataFrame. We want to merge based on the state/region column of pop, and the abbreviation column of abbrevs. We'll use how='outer' to make sure no data is thrown away due to mismatched labels.

In [67]:
merged = pd.merge(pop, abbrev, how='outer',
                 left_on='state/region', right_on='abbreviation')

merged=merged.drop('abbreviation', 1) #drop duplicate info

merged.head()


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [68]:
merged.isnull().any()


state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the population info is null; let's figure out which these are!

In [69]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


It appears that all the null population values are from Puerto Rico (PR) prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new state entries are also null, which means that there was no corresponding entry in the abbrevs key! Let's figure out which regions lack this match:

In [70]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [71]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()


state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more missing values or NaNs for state.

Now we can merge the result with the area data using a similar procedure. Examining our results, we will want to join on the state column in both:

In [72]:
final = pd.merge(merged, area, on='state', how='left')
final.head()


,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [73]:
final.isnull().any()


state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [74]:
final['state'][final['area (sq. mi)'].isnull()].unique()


array(['United States'], dtype=object)

In [75]:

final.dropna(inplace=True)
final.head()


,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2010, and the total population. We'll use the query( ) function to do this quickly.

In [76]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()


,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let's compute the population density and display it in order. We'll start by re-indexing our data on the state, and then compute the result:

In [77]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']


In [78]:
density.sort_values(ascending=False, inplace=True)
density.head()


state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

The result is ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.  We can see that by far the densest region in this dataset is Washington, DC; among states, the densest is New Jersey.

In [79]:
density.tail()


state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

We can check the lowest density by looking at the end of the list.  Alaska appears to have the lowest population density.